In [1]:
import tensorflow as tf
import os, argparse, pickle
import numpy as np
import efficientnet.tfkeras as efn
from tqdm import tqdm

In [2]:
def get_args():
    parser = argparse.ArgumentParser()

#     parser.add_argument('--dataset', type=str, default='cifar10',
#                         help='dataset used for training (e.g. cifar10, cifar100, svhn, svhn+extra)')

    parser.add_argument('--epochs', type=int, default=1024, help='number of epochs, (default: 1024)')
    parser.add_argument('--batch_size',  type=int, default=128, help='examples per batch (default: 64)')
    parser.add_argument('--learning_rate', type=float, default=0.256, help='learning_rate, (default: 0.256)')
    parser.add_argument('--lr_decay', type=float, default=0.94, help='learning rate decay, (default: 0.94)')
    parser.add_argument('--tensorboard', action='store_true', help='enable tensorboard visualization')
    parser.add_argument('--resume', action='store_true', help='whether to restore from previous training runs')
    parser.add_argument('--gpus', type=str, default='-1', help='set gpu numbers')

    return parser.parse_known_args(args='')

def scheduler(epoch, lr):
    if not(epoch != 0 and epoch % 2 == 0):
        return lr
    else:
        return lr * 0.94


In [32]:
class get_generator():
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        
    def __call__(self):
        for _x, _y in zip(self.x, self.y):
            x = tf.image.resize(_x, self.shape[:2])
            if _y == -1:
                y = _y + 11
            else:
                y = _y / 20
            yield (x, y)

In [33]:
def main():
    config = get_args()[0]
    os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    # os.environ['CUDA_VISIBLE_DEVICES'] = config.gpus
    abspath = './drive/My Drive/Colab Notebooks'
    datapath = os.path.join(abspath, 'dataset/interspeech20/acoustic')
    trainx = pickle.load(open(os.path.join(datapath, 'trainset_x.pickle'), 'rb'))
    trainy = pickle.load(open(os.path.join(datapath, 'trainset_y.pickle'), 'rb'))
    testx = pickle.load(open(os.path.join(datapath, 'testset_x.pickle'), 'rb'))
    testy = pickle.load(open(os.path.join(datapath, 'trainset_y.pickle'), 'rb'))
    
    shape = 0
    for data in trainx:
        if data.shape[0] > shape:
            shape = data.shape[0]
    for data in testx:
        if data.shape[0] > shape:
            shape = data.shape[0]
    # shape = (shape // (len(trainx) + len(testx)), trainx[0].shape[1], trainx[0].shape[2])
    shape = (shape, trainx[0].shape[1], trainx[0].shape[2])
            
    name = f'b0_RMS_{config.learning_rate}'
    
    tensorboard_path = './tensorboard_log/'+name
    model_save_path = './model_save/'+name
    if not os.path.exists(tensorboard_path):
        os.makedirs(tensorboard_path)
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    trainx = np.concatenate(np.array([tf.image.resize_with_pad(i,shape[0],shape[1])[tf.newaxis, ...].numpy() for i in trainx]))
    testx = np.concatenate(np.array([tf.image.resize_with_pad(i,shape[0],shape[1])[tf.newaxis, ...].numpy() for i in testx]))
    trainy = np.array([i / 20 if i != -1 else i + 11 for i in trainy])
    testy = np.array([i / 20 if i != -1 else i + 11 for i in testy])
    train_dataset = tf.data.Dataset.from_tensor_slices((trainx, trainy)).shuffle(len(trainx)).batch(config.batch_size).prefetch(AUTOTUNE)
    validation_dataset = tf.data.Dataset.from_tensor_slices((testx, testy)).batch(config.batch_size)
    # train_generator = get_generator(trainx, trainy, shape)
    # test_generator = get_generator(testx, testy, shape)
    # train_dataset = tf.data.Dataset.from_generator(train_generator, (tf.float32, tf.int32), (tf.TensorShape(list(shape)), tf.TensorShape([]))).shuffle(len(trainx)).padded_batch(config.batch_size).prefetch(AUTOTUNE)
    # validation_dataset = tf.data.Dataset.from_generator(test_generator, (tf.float32, tf.int32), (tf.TensorShape(list(shape)), tf.TensorShape([]))).shuffle(len(testx)).batch(config.batch_size)

    optimizer = tf.keras.optimizers.RMSprop(lr=config.learning_rate, momentum=0.9)
    # with strategy.scope():
    model = get_model(shape)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=['acc'])
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_path, histogram_freq=1)
    lrscheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_acc', patience=10, verbose=0, mode='auto', restore_best_weights=True
    )
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        model_save_path, verbose=0,
        save_freq='epoch',
        monitor='val_acc'
    )
    
    model.summary()
    model.fit(train_dataset,epochs=config.epochs, validation_data=validation_dataset, callbacks=[
        tensorboard_callback,
        earlystop,
        cp_callback
    ])

In [34]:
def get_model(input_shape):
    inputs = tf.keras.layers.Input(input_shape)
    efficientnet = efn.EfficientNetB0(weights=None,input_shape=input_shape,include_top=False)(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(efficientnet)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(11,activation='softmax')(x)
    return tf.keras.Model(inputs=inputs, outputs=out)

In [35]:
if __name__ == '__main__':
#     model = get_model((None,120,4))
#     model.summary()
    main()

/root/anaconda3/envs/ten2.3/lib/python3.7/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  str(input_shape[-1]) + ' input channels.')


Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 220, 120, 4)]     0         
_________________________________________________________________
efficientnet-b0 (Functional) (None, 7, 4, 1280)        4049852   
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 4, 1280)        1639680   
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 4, 1280)        5120      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 2, 1280)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 11)              

0.0
2.0
8.0
7.0
6.0
0.0
2.0
5.0
2.0
5.0
4.0
5.0
3.0
5.0
5.0
2.0
5.0
8.0
4.0
3.0
8.0
4.0
8.0
3.0
8.0
1.0
7.0
2.0
3.0
1.0
7.0
8.0
8.0
4.0
3.0
1.0
9.0
1.0
5.0
9.0
6.0
3.0
4.0
0.0
9.0
8.0
0.0
6.0
5.0
4.0
9.0
3.0
0.0
5.0
6.0
3.0
6.0
7.0
2.0
0.0
3.0
4.0
7.0
4.0
6.0
5.0
3.0
8.0
6.0
7.0
3.0
2.0
8.0
5.0
7.0
8.0
4.0
7.0
4.0
2.0
1.0
0.0
0.0
9.0
6.0
4.0
5.0
3.0
7.0
4.0
1.0
9.0
8.0
0.0
4.0
6.0
8.0
2.0
5.0
7.0
6.0
1.0
0.0
8.0
9.0
8.0
4.0
5.0
7.0
6.0
4.0
9.0
0.0
0.0
9.0
5.0
4.0
9.0
3.0
4.0
8.0
1.0
6.0
6.0
2.0
6.0
4.0
2.0
9.0
6.0
0.0
9.0
4.0
4.0
6.0
6.0
3.0
7.0
6.0
2.0
0.0
3.0
0.0
2.0
2.0
8.0
7.0
1.0
3.0
5.0
3.0
4.0
5.0
1.0
4.0
2.0
9.0
9.0
1.0
6.0
0.0
1.0
5.0
7.0
7.0
0.0
3.0
1.0
9.0
2.0
4.0
9.0
1.0
6.0
1.0
1.0
7.0
8.0
2.0
6.0
3.0
7.0
3.0
9.0
1.0
5.0
1.0
9.0
6.0
0.0
6.0
2.0
8.0
9.0
6.0
2.0
6.0
8.0
6.0
7.0
6.0
3.0
1.0
3.0
6.0
4.0
8.0
8.0
6.0
8.0
5.0
2.0
4.0
1.0
0.0
0.0
2.0
6.0
4.0
7.0
5.0
1.0
1.0
5.0
6.0
0.0
8.0
8.0
2.0
2.0
8.0
2.0
5.0
4.0
9.0
4.0
6.0
5.0
8.0
4.0
7.0
6.0
1.0
2.0
3.0
0.0
1.0
5.0
4.0
9.0


5.0
4.0
2.0
9.0
3.0
6.0
2.0
1.0
9.0
5.0
2.0
1.0
7.0
2.0
4.0
9.0
5.0
4.0
2.0
9.0
0.0
0.0
5.0
1.0
5.0
0.0
6.0
4.0
8.0
6.0
2.0
4.0
7.0
6.0
7.0
8.0
4.0
4.0
4.0
5.0
3.0
4.0
6.0
1.0
6.0
6.0
2.0
0.0
0.0
4.0
3.0
3.0
2.0
3.0
0.0
2.0
3.0
5.0
9.0
2.0
2.0
1.0
4.0
5.0
0.0
0.0
3.0
1.0
7.0
7.0
5.0
2.0
3.0
1.0
1.0
9.0
5.0
7.0
7.0
9.0
9.0
9.0
1.0
7.0
5.0
2.0
1.0
9.0
7.0
3.0
1.0
4.0
5.0
8.0
2.0
9.0
0.0
3.0
8.0
1.0
9.0
8.0
1.0
0.0
3.0
8.0
1.0
1.0
2.0
1.0
5.0
0.0
1.0
4.0
3.0
5.0
2.0
4.0
0.0
5.0
4.0
8.0
5.0
1.0
3.0
8.0
7.0
2.0
0.0
2.0
3.0
0.0
6.0
5.0
8.0
6.0
9.0
1.0
9.0
3.0
5.0
5.0
5.0
1.0
2.0
1.0
7.0
2.0
2.0
1.0
6.0
9.0
7.0
3.0
8.0
5.0
8.0
4.0
7.0
0.0
3.0
9.0
7.0
5.0
2.0
2.0
9.0
7.0
0.0
4.0
5.0
6.0
0.0
6.0
9.0
3.0
3.0
8.0
3.0
2.0
7.0
1.0
2.0
1.0
7.0
0.0
6.0
2.0
5.0
8.0
4.0
7.0
1.0
4.0
8.0
3.0
6.0
0.0
5.0
2.0
0.0
3.0
9.0
2.0
6.0
4.0
1.0
7.0
2.0
9.0
7.0
5.0
1.0
9.0
3.0
6.0
8.0
6.0
2.0
8.0
8.0
5.0
0.0
7.0
5.0
4.0
7.0
6.0
0.0
5.0
6.0
9.0
3.0
3.0
3.0
0.0
4.0
9.0
8.0
6.0
1.0
7.0
1.0
1.0
5.0
1.0
0.0
6.0
7.0
8.0


10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
8.0
8.0
5.0
4.0
2.0
0.0
3.0
9.0
5.0
5.0
9.0
7.0
3.0
2.0
2.0
8.0
1.0
7.0
4.0
4.0
3.0
5.0
3.0
3.0
4.0
5.0
8.0
3.0
8.0
4.0
4.0
4.0
7.0
2.0
5.0
3.0
9.0
2.0
2.0
4.0
3.0
8.0
7.0
3.0
0.0
0.0
4.0
0.0
4.0
3.0
3.0
5.0
9.0
8.0
2.0
7.0
2.0
9.0
0.0
4.0
6.0
5.0
2.0
6.0
0.0
8.0
2.0
4.0
3.0
0.0
7.0
2.0
5.0
4.0
4.0
1.0
6.0
6.0
1.0
6.0
5.0
3.0
0.0
4.0
3.0
8.0
7.0
6.0
9.0
7.0
4.0
4.0
3.0
4.0
8.0
2.0
6.0
5.0
0.0
4.0
1.0
9.0
5.0
9.0
3.0
4.0
1.0
8.0
1.0
8.0
7.0
4.0
6.0
0.0
2.0
8.0
9.0
2.0
0.0
2.0
1.0
8.0
3.0
5.0
7.0
8.0
8.0
1.0
3.0
4

...

9.0
5.0
0.0
6.0
7.0
3.0
8.0
0.0
3.0
2.0
8.0
1.0
6.0
3.0
4.0
5.0
0.0
2.0
5.0
9.0
5.0
7.0
0.0
2.0
3.0
2.0
3.0
0.0
4.0
3.0
6.0
6.0
8.0
0.0
9.0
1.0
2.0
1.0
7.0
6.0
9.0
5.0
9.0
7.0
1.0
6.0
3.0
8.0
2.0
1.0
3.0
7.0
8.0
9.0
9.0
8.0
7.0
5.0
9.0
8.0
8.0
9.0
1.0
7.0
6.0
3.0
2.0
5.0
1.0
1.0
2.0
9.0
2.0
8.0
0.0
3.0
9.0
1.0
0.0
8.0
9.0
6.0
8.0
7.0
0.0
7.0
4.0
7.0
7.0
4.0
9.0
1.0
0.0
5.0
5.0
5.0
6.0
8.0
4.0
2.0
5.0
7.0
9.0
3.0
6.0
0.0
5.0
0.0
0.0
5.0
6.0
5.0
6.0
0.0
5.0
4.0
7.0
0.0
0.0
3.0
8.0
2.0
7.0
6.0
2.0
2.0
0.0
5.0
1.0
2.0
6.0
0.0
8.0
9.0
0.0
1.0
4.0
7.0
5.0
7.0
4.0
0.0
1.0
8.0
2.0
8.0
4.0
3.0
7.0
0.0
5.0
5.0
8.0
3.0
9.0
0.0
8.0
6.0
2.0
5.0
8.0
0.0
7.0
9.0
5.0
6.0
4.0
2.0
7.0
7.0
9.0
0.0
5.0
7.0
4.0
6.0
3.0
0.0
6.0
0.0
6.0
3.0
5.0
5.0
1.0
4.0
9.0
4.0
1.0
4.0
8.0
6.0
9.0
0.0
3.0
2.0
9.0
9.0
1.0
8.0
5.0
4.0
9.0
6.0
8.0
8.0
0.0
8.0
1.0
7.0
3.0
5.0
5.0
9.0
4.0
7.0
7.0
6.0
7.0
1.0
1.0
6.0
3.0
9.0
1.0
1.0
6.0
4.0
9.0
3.0
8.0
3.0
8.0
4.0
8.0
1.0
0.0
0.0
1.0
1.0
8.0
2.0
9.0
6.0
6.0
3.0
6.0
5.0
0.0
1.0


9.0
1.0
9.0
8.0
1.0
4.0
8.0
4.0
7.0
6.0
6.0
6.0
2.0
7.0
5.0
1.0
3.0
6.0
7.0
1.0
2.0
8.0
3.0
4.0
9.0
5.0
3.0
2.0
8.0
9.0
9.0
1.0
9.0
2.0
2.0
5.0
9.0
9.0
3.0
4.0
8.0
9.0
5.0
2.0
1.0
2.0
2.0
0.0
0.0
3.0
9.0
4.0
4.0
0.0
8.0
1.0
4.0
7.0
9.0
9.0
0.0
3.0
6.0
6.0
9.0
5.0
1.0
6.0
9.0
3.0
5.0
4.0
4.0
2.0
1.0
9.0
1.0
8.0
7.0
9.0
3.0
9.0
2.0
2.0
8.0
5.0
9.0
1.0
8.0
0.0
1.0
0.0
4.0
0.0
7.0
4.0
9.0
9.0
2.0
2.0
6.0
9.0
4.0
0.0
2.0
9.0
9.0
9.0
3.0
6.0
2.0
7.0
5.0
6.0
5.0
2.0
3.0
1.0
5.0
3.0
0.0
7.0
0.0
2.0
9.0
4.0
3.0
6.0
2.0
2.0
0.0
4.0
8.0
7.0
2.0
4.0
3.0
0.0
4.0
0.0
3.0
4.0
4.0
1.0
1.0
9.0
0.0
8.0
9.0
8.0
9.0
5.0
4.0
6.0
2.0
2.0
7.0
9.0
3.0
0.0
6.0
0.0
8.0
5.0
8.0
1.0
7.0
1.0
9.0
0.0
5.0
6.0
3.0
3.0
7.0
1.0
8.0
5.0
2.0
3.0
6.0
6.0
7.0
9.0
4.0
0.0
0.0
1.0
3.0
8.0
6.0
4.0
0.0
5.0
5.0
3.0
6.0
0.0
0.0
2.0
8.0
8.0
9.0
8.0
6.0
7.0
5.0
3.0
4.0
6.0
6.0
1.0
8.0
5.0
4.0
1.0
6.0
0.0
7.0
7.0
1.0
0.0
0.0
8.0
6.0
6.0
4.0
2.0
1.0
1.0
9.0
1.0
9.0
3.0
2.0
2.0
1.0
6.0
8.0
3.0
5.0
3.0
8.0
3.0
0.0
1.0
8.0
1.0
8.0
8.0


6.0
5.0
6.0
0.0
9.0
2.0
0.0
4.0
0.0
3.0
0.0
3.0
9.0
0.0
6.0
7.0
6.0
6.0
4.0
2.0
5.0
1.0
4.0
1.0
7.0
6.0
8.0
5.0
1.0
7.0
4.0
9.0
6.0
3.0
9.0
5.0
8.0
3.0
4.0
8.0
1.0
6.0
5.0
1.0
0.0
3.0
7.0
1.0
5.0
2.0
0.0
4.0
6.0
3.0
7.0
1.0
6.0
7.0
1.0
7.0
4.0
3.0
4.0
5.0
9.0
8.0
5.0
4.0
7.0
6.0
9.0
4.0
0.0
7.0
0.0
1.0
8.0
9.0
5.0
1.0
3.0
9.0
7.0
9.0
9.0
7.0
6.0
6.0
1.0
0.0
0.0
8.0
4.0
8.0
7.0
2.0
8.0
3.0
5.0
9.0
0.0
2.0
2.0
9.0
1.0
7.0
7.0
7.0
4.0
8.0
0.0
3.0
5.0
0.0
4.0
5.0
0.0
5.0
5.0
0.0
8.0
3.0
6.0
9.0
6.0
3.0
9.0
1.0
4.0
8.0
0.0
2.0
1.0
4.0
6.0
9.0
7.0
2.0
3.0
8.0
0.0
4.0
5.0
0.0
0.0
5.0
0.0
3.0
5.0
4.0
8.0
9.0
8.0
7.0
1.0
9.0
6.0
1.0
3.0
2.0
7.0
9.0
3.0
6.0
5.0
9.0
5.0
9.0
4.0
7.0
9.0
8.0
7.0
1.0
6.0
4.0
1.0
4.0
3.0
0.0
9.0
1.0
2.0
8.0
5.0
9.0
6.0
3.0
4.0
2.0
2.0
6.0
8.0
0.0
8.0
6.0
9.0
2.0
5.0
0.0
1.0
4.0
0.0
0.0
9.0
9.0
9.0
1.0
4.0
8.0
7.0
4.0
2.0
3.0
8.0
1.0
7.0
1.0
1.0
7.0
6.0
4.0
9.0
9.0
5.0
1.0
8.0
4.0
8.0
8.0
6.0
8.0
5.0
9.0
4.0
7.0
1.0
0.0
6.0
5.0
7.0
0.0
6.0
0.0
1.0
7.0
0.0
7.0
1.0
1.0


7.0
2.0
3.0
2.0
1.0
1.0
7.0
7.0
5.0
4.0
9.0
9.0
8.0
1.0
7.0
4.0
6.0
0.0
9.0
8.0
1.0
2.0
4.0
8.0
4.0
5.0
2.0
9.0
8.0
4.0
1.0
8.0
5.0
3.0
3.0
5.0
2.0
6.0
7.0
8.0
2.0
2.0
9.0
2.0
5.0
1.0
0.0
4.0
8.0
2.0
4.0
3.0
9.0
9.0
7.0
0.0
1.0
7.0
4.0
8.0
7.0
0.0
7.0
4.0
4.0
1.0
9.0
6.0
9.0
3.0
6.0
3.0
4.0
4.0
5.0
3.0
9.0
6.0
3.0
1.0
7.0
9.0
5.0
3.0
8.0
2.0
1.0
5.0
7.0
3.0
8.0
0.0
4.0
0.0
2.0
7.0
9.0
5.0
7.0
8.0
5.0
2.0
0.0
2.0
0.0
7.0
3.0
7.0
9.0
2.0
2.0
6.0
5.0
7.0
1.0
4.0
9.0
2.0
7.0
2.0
7.0
4.0
9.0
3.0
2.0
6.0
6.0
9.0
4.0
7.0
6.0
5.0
2.0
4.0
9.0
0.0
6.0
6.0
6.0
2.0
6.0
2.0
9.0
8.0
2.0
8.0
9.0
5.0
6.0
5.0
8.0
0.0
2.0
3.0
0.0
2.0
1.0
8.0
2.0
7.0
2.0
5.0
4.0
2.0
5.0
6.0
8.0
4.0
7.0
4.0
4.0
3.0
9.0
8.0
4.0
6.0
9.0
2.0
2.0
6.0
4.0
6.0
7.0
8.0
4.0
4.0
8.0
7.0
3.0
5.0
7.0
7.0
5.0
5.0
9.0
8.0
2.0
2.0
8.0
7.0
3.0
4.0
3.0
4.0
7.0
2.0
0.0
6.0
6.0
7.0
3.0
0.0
8.0
9.0
5.0
9.0
9.0
1.0
4.0
5.0
8.0
1.0
0.0
3.0
9.0
3.0
3.0
7.0
3.0
6.0
2.0
5.0
9.0
8.0
7.0
4.0
9.0
8.0
7.0
1.0
4.0
7.0
8.0
2.0
5.0
9.0
6.0
7.0
8.0
8.0


KeyboardInterrupt: 